In [1]:
import os
import pandas as pd

# baixar o conjunto de dados "tiny" do MS MARCO na tarefa doc2query
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

# carregar os dados do arquivo tsv em um DataFrame do pandas
df = pd.read_csv('msmarco_triples.train.tiny.tsv', delimiter='\t', header=None, names=['query', 'passage', 'label'])

# extrair os pares (query, passagem relevante) e salvá-los em um novo arquivo tsv
df[['query', 'passage']].to_csv('msmarco_train_pairs.tsv', sep='\t', index=False, header=None)

# baixar o índice invertido do Trec-covid no pyserini (beir-v1.0.0-trec-covid-flat)
!wget https://huggingface.co/BeIR/trec-covid/raw/main/beir-v1.0.0-trec-covid-flat.zip
!unzip beir-v1.0.0-trec-covid-flat.zip -d beir-v1.0.0-trec-covid-flat

# carregar o corpus do Trec-covid na HF
from datasets import load_dataset

corpus = load_dataset('BeIR/trec-covid', 'paragraphs')['train']

# carregar o modelo T5-base do Hugging Face
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# definir uma função para gerar a query a partir de uma passagem
def generate_query(passage):
    input_text = "doc2query: " + passage
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

# para cada documento na coleção Trec-covid, gerar uma ou mais queries usando o doc2query e concatená-las ao documento original para criar um documento "expandido". Este documento expandido é indexado usando o índice invertido do Trec-covid no pyserini.
from pyserini.index import IndexWriter

# criar um índice invertido para os documentos expandidos
writer = IndexWriter('beir-v1.0.0-trec-covid-flat/expanded_index', 'lucene-index')

for i, document in enumerate(corpus):
    # gerar uma ou mais queries usando o doc2query
    queries = [generate_query(passage) for passage in document['paragraphs']]
    # concatenar as queries ao documento original para criar um documento "expandido"
    expanded_document = ' '.join([document['title']] + queries + [passage for passage in document['paragraphs']])
    # adicionar o documento expandido ao índice invertido
    writer.add_document(docid=str(i), contents=expanded_document)

# finalizar a escrita do índice invertido
writer.commit()

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'msmarco_triples.train.tiny.tsv'